In [1]:
!pip install transformers

     |████████████████████████████████| 1.4MB 7.7MB/s 
     |████████████████████████████████| 890kB 29.2MB/s 
     |████████████████████████████████| 2.9MB 44.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1ed65f109a13ea7fd43396507058030ec5d6c6ed5f99b3bccfe23047c36589b8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import json

In [3]:
context = 'Following the Cretaceous–Paleogene extinction event, the extinction of the dinosaurs and the wetter climate may have allowed the tropical rainforest to spread out across the continent. From 66–34 Mya, the rainforest extended as far south as 45°. Climate fluctuations during the last 34 million years have allowed savanna regions to expand into the tropics. During the Oligocene, for example, the rainforest spanned a relatively narrow band. It expanded again during the Middle Miocene, then retracted to a mostly inland formation at the last glacial maximum. However, the rainforest still managed to thrive during these glacial periods, allowing for the survival and evolution of a broad diversity of species.'
question = 'Which type of climate may have allowed the rainforest to spread across the continent?'
true_answer = ['the wetter climate may have allowed the tropical rainforest to spread out across the continent', 'wetter']

In [4]:
#Model
bert_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [12]:
encoding = bert_tokenizer.encode_plus(text=question,text_pair=context)

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = bert_tokenizer.convert_ids_to_tokens(inputs) #input tokens
outputs = bert_model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

start_index = torch.argmax(outputs.start_logits)

end_index = torch.argmax(outputs.end_logits)

bert_answer = ''.join(tokens[start_index:end_index+1]).replace('#','')
bert_answer

'wetter'

In [11]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [13]:
em_score = max((compute_exact_match(bert_answer, a)) for a in true_answer)
f1_score = max((compute_f1(bert_answer, a)) for a in true_answer)

print(em_score)
print(f1_score)

1
1.0


Albert

In [14]:
from transformers import AlbertTokenizer, AlbertForQuestionAnswering
#!pip install sentencepiece

In [15]:
import sentencepiece as spm
albert_tokenizer = AlbertTokenizer.from_pretrained('ahotrod/albert_xxlargev1_squad2_512')
albert_model = AlbertForQuestionAnswering.from_pretrained('ahotrod/albert_xxlargev1_squad2_512', return_dict=True)

In [19]:
encoding = albert_tokenizer.encode_plus(text=question,text_pair=context)

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = albert_tokenizer.convert_ids_to_tokens(inputs) #input tokens
outputs = albert_model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

start_index = torch.argmax(outputs.start_logits)

end_index = torch.argmax(outputs.end_logits)

albert_answer = ''.join(tokens[start_index:end_index+1]).replace('#','').replace('▁','')
albert_answer

'tropical'

In [20]:
em_score = max((compute_exact_match(albert_answer, a)) for a in true_answer)
f1_score = max((compute_f1(albert_answer, a)) for a in true_answer)

print(em_score)
print(f1_score)

0
0.15384615384615385


Electra

In [21]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
electra_model = ElectraForQuestionAnswering.from_pretrained('google/electra-small-discriminator')

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_output

In [22]:
encoding = electra_tokenizer.encode_plus(text=question,text_pair=context)

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = electra_tokenizer.convert_ids_to_tokens(inputs) #input tokens
outputs = electra_model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

start_index = torch.argmax(outputs.start_logits)

end_index = torch.argmax(outputs.end_logits)

electra_answer = ''.join(tokens[start_index:end_index+1]).replace('#','').replace('▁','')
electra_answer

''

In [24]:
em_score = max((compute_exact_match(electra_answer, a)) for a in true_answer)
f1_score = max((compute_f1(electra_answer, a)) for a in true_answer)

print(em_score)
print(f1_score)

0
0
